In [ ]:
# Importation des bibliothèques nécessaires
import requests
from bs4 import BeautifulSoup
import json

# Fonction pour récupérer les données du Nasdaq
def scrape_nasdaq_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    tables = soup.find_all('table', class_='wikitable')
    
    data = {
        "largest_gains": [],
        "largest_losses": []
    }

    for i, table in enumerate(tables[:2]):  # We only need the first two tables
        rows = table.find_all('tr')[1:]  # Skip the header row
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 5:
                entry = {
                    "rank": cells[0].text.strip(),
                    "date": cells[1].text.strip(),
                    "close": cells[2].text.strip(),
                    "net_change": cells[3].text.strip(),
                    "percent_change": cells[4].text.strip()
                }
                if i == 0:
                    data["largest_gains"].append(entry)
                else:
                    data["largest_losses"].append(entry)

    return data

# Fonction pour sauvegarder les données en JSON
def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"Data has been saved to {filename}")

# URL de la page Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_largest_daily_changes_in_the_Nasdaq_Composite"

# Récupération des données
nasdaq_data = scrape_nasdaq_data(url)

# Affichage des premières entrées pour vérification
print("Largest gains (first 3):")
print(json.dumps(nasdaq_data["largest_gains"][:3], indent=2))

print("\nLargest losses (first 3):")
print(json.dumps(nasdaq_data["largest_losses"][:3], indent=2))

# Sauvegarde des données (optionnel)
# Décommentez la ligne suivante si vous voulez sauvegarder les données
# save_to_json(nasdaq_data, "nasdaq_largest_changes.json")